In [5]:
##################### rascunho requests que deu
import requests

# Replace with your RapidAPI key
RAPIDAPI_KEY = api_key
icao_code = 'KJFK'  # Replace with the desired ICAO code

# Set up the API endpoint and headers
url = f'https://airport-info.p.rapidapi.com/airport?icao={icao_code}'
headers = {
    'X-RapidAPI-Host': 'airport-info.p.rapidapi.com',
    'X-RapidAPI-Key': RAPIDAPI_KEY
}

# Make an HTTP GET request to the API
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(data)
    # Now you can process the data or create a PySpark DataFrame as needed.
else:
    print(f"Failed to retrieve data from the API. Status code: {response.status_code}")

{'id': 3406, 'iata': 'JFK', 'icao': 'KJFK', 'name': 'John F. Kennedy International Airport', 'location': 'New York City, New York, United States', 'street_number': '', 'street': '', 'city': '', 'county': 'Queens County', 'state': 'New York', 'country_iso': 'US', 'country': 'United States', 'postal_code': '11430', 'phone': '+1 718-244-4444', 'latitude': 40.64131, 'longitude': -73.77814, 'uct': -240, 'website': 'https://www.jfkairport.com/'}


In [4]:
import http.client
conn = http.client.HTTPSConnection("airport-info.p.rapidapi.com")
headers = {
    'x-rapidapi-key': api_key,
    'x-rapidapi-host': "airport-info.p.rapidapi.com"
    }
conn.request("GET", "/airport?icao=KJFK", headers=headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"id":3406,"iata":"JFK","icao":"KJFK","name":"John F. Kennedy International Airport","location":"New York City, New York, United States","street_number":"","street":"","city":"","county":"Queens County","state":"New York","country_iso":"US","country":"United States","postal_code":"11430","phone":"+1 718-244-4444","latitude":40.64131,"longitude":-73.77814,"uct":-240,"website":"https://www.jfkairport.com/"}



tentativa com tabela dimensao

In [6]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=32f0d78bf12a8d87e9c3c5b89492a1d9517b88882b5300cb05a0af2c54e09858
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
!pip install python-dotenv


In [34]:
import os
import sys
import re
import json
from dotenv import load_dotenv
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


# Access the API key
api_key = os.getenv("API_KEY")

# Initialize SparkSession
spark = SparkSession.builder.appName("API Data Extraction").getOrCreate()

# Step 1: Read data from the Parquet file
df_vra = spark.read.parquet("/content/VRA")
df_vra.show(5)

# Step 2: Read data from the DataFrame

unique_codes = (
    df_vra.select("icao_aerodromo_origem")
    .union(df_vra.select("icao_aerodromo_destino"))
    .distinct()
)
# unique_codes = unique_codes.withColumnRenamed("icao_aerodromo_origem", "icao")
# unique_codes.show(5)

df_vra_origem = df_vra.select("icao_aerodromo_origem").distinct()
print()
df_vra_destino = df_vra.select("icao_aerodromo_destino").distinct()


unique_codes = unique_codes.withColumnRenamed("icao_aerodromo_origem", "icao")
unique_codes = df_vra_origem.union(df_vra_destino)
unique_codes.show()

+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|   chegada_prevista|       chegada_real|codigo_autorizacao|codigo_justificativa|codigo_tipo_linha|icao_aerodromo_destino|icao_aerodromo_origem|icao_empresa_aerea|numero_voo|   partida_prevista|       partida_real|situacao_voo|
+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-11-12 08:30:00|2021-11-12 08:30:00|                 0|                 N/A|                X|                  KORD|                 SBGR|               UAL|       844|2021-11-11 22:00:00|2021-11-11 22:14:00|   REALIZADO|
|2021-11-15 08:30:00|2021-11-15 08:30:00|                 0|                 N/A|       

In [35]:
# Define the schema for the DataFrame
schema = StructType([
    StructField("icao_code", StringType(), True),
    StructField("api_response", StringType(), True)
])

In [43]:
# Function to fetch data from the API
def fetch_data(code, use_iata=False, use_icao=False):
    api_url = f'https://airport-info.p.rapidapi.com/airport?icao={icao_code}'
    headers = {
        "X-RapidAPI-Host": "active-api.p.rapidapi.com",
        "X-RapidAPI-Key": api_key,  # Replace with your RapidAPI key
    }
    params = {}

    # if use_iata:
    #     params["iata"] = code

    if use_icao:
        params["icao"] = code

    response = requests.get(api_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [37]:
# Step 2: Send requests to the API for each unique code and create a new DataFrame
#api_responses = unique_codes.rdd.map(lambda row: (row[0], fetch_data(row[0], use_iata=True, use_icao=True))).toDF(schema)
api_responses = unique_codes.rdd.map(lambda row: (row[0], fetch_data(row[0], use_icao=True))).toDF(schema)

In [38]:
api_responses.show()

+---------+------------+
|icao_code|api_response|
+---------+------------+
|     LFPG|        NULL|
|     SBPS|        NULL|
|     KATL|        NULL|
|     SBIH|        NULL|
|     EDDK|        NULL|
|     SSSC|        NULL|
|     SBSJ|        NULL|
|     OTHH|        NULL|
|     SBCH|        NULL|
|     SCCF|        NULL|
|     FYWH|        NULL|
|     SBCR|        NULL|
|     UCFM|        NULL|
|     SNEE|        NULL|
|     SDWQ|        NULL|
|     PANC|        NULL|
|     ZHHH|        NULL|
|     SBUG|        NULL|
|     SWPH|        NULL|
|     OMAA|        NULL|
+---------+------------+
only showing top 20 rows



In [26]:

# Step 3: Create a DataFrame from the successful API responses
api_dataframe = api_responses.filter(api_responses["api_response"].isNotNull())

In [27]:
api_dataframe.show(truncate=False)

+---------+------------+
|icao_code|api_response|
+---------+------------+
+---------+------------+



#Teste 2

In [4]:
spark.stop()

In [42]:
import http.client
conn = http.client.HTTPSConnection("airport-info.p.rapidapi.com")
headers = {
    'x-rapidapi-key': api_key,
    'x-rapidapi-host': "airport-info.p.rapidapi.com"
    }
conn.request("GET", "/airport?icao={icao_code}", headers=headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

In [45]:
import http.client

# Define a function to fetch data from the API
def fetch_data(icao_code):
    # Define the connection to the API
    conn = http.client.HTTPSConnection("airport-info.p.rapidapi.com")

    # Define headers
    headers = {
        'x-rapidapi-key': api_key,
        'x-rapidapi-host': "airport-info.p.rapidapi.com"
    }

    # Build the API request URL with the icao_code
    url = f"/airport?icao={icao_code}"

    # Send a GET request to the API
    conn.request("GET", url, headers=headers)

    # Get the response
    res = conn.getresponse()

    if res.status == 200:
        # Read and decode the response data
        data = res.read().decode("utf-8")
        return data
    else:
        return None

# Example usage
icao_code = "KJFK"  # Replace with the desired icao_code
api_response = fetch_data(icao_code)

if api_response is not None:
    print(api_response)
else:
    print("API request failed.")


{"id":3406,"iata":"JFK","icao":"KJFK","name":"John F. Kennedy International Airport","location":"New York City, New York, United States","street_number":"","street":"","city":"","county":"Queens County","state":"New York","country_iso":"US","country":"United States","postal_code":"11430","phone":"+1 718-244-4444","latitude":40.64131,"longitude":-73.77814,"uct":-240,"website":"https://www.jfkairport.com/"}



In [15]:
import http.client

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


# Define a function to fetch data from the API
def fetch_data(icao_code):
    # Define the connection to the API
    conn = http.client.HTTPSConnection("airport-info.p.rapidapi.com")

    # Define headers
    headers = {
        'x-rapidapi-key': api_key,
        'x-rapidapi-host': "airport-info.p.rapidapi.com"
    }

    # Build the API request URL with the icao_code
    url = f"/airport?icao={icao_code}"

    # Send a GET request to the API
    conn.request("GET", url, headers=headers)

    # Get the response
    res = conn.getresponse()

    if res.status == 200:
        # Read and decode the response data
        data = res.read().decode("utf-8")
        return data
    else:
        return None



# Initialize SparkSession
spark = SparkSession.builder.appName("API Data Extraction").getOrCreate()

# Step 1: Read data from the Parquet file
df_vra = spark.read.parquet("/content/VRA")
df_vra.show(5)

# Step 2: Read data from the DataFrame

unique_codes = (
    df_vra.select("icao_aerodromo_origem")
    .union(df_vra.select("icao_aerodromo_destino"))
    .distinct()
)

# Rename the column to "icao"
unique_codes = unique_codes.withColumnRenamed("icao_aerodromo_origem", "icao")

# Define a schema that covers all possible keys in the JSON-like responses
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("iata", StringType(), True),
    StructField("icao", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("street_number", StringType(), True),
    StructField("street", StringType(), True),
    StructField("city", StringType(), True),
    StructField("county", StringType(), True),
    StructField("state", StringType(), True),
    StructField("country_iso", StringType(), True),
    StructField("country", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("uct", IntegerType(), True),
    StructField("website", StringType(), True)
])

# Create an empty DataFrame to hold the API responses
api_dataframe = spark.createDataFrame([], schema=schema)

# Fetch data for each unique code and add it to the DataFrame
for row in unique_codes.collect():
    icao_code = row["icao"]
    api_response = fetch_data(icao_code)
    if api_response:
        api_dataframe = api_dataframe.union(spark.createDataFrame([[None, None, icao_code, None, None, None, None, None, None, None, None, None, None, None, None, None, None, api_response]], schema=schema))

# Add a column to indicate the source icao_code for each record
api_dataframe = api_dataframe.withColumn("icao_code_source", lit("icao_code"))

# Print the DataFrame to view the results
api_dataframe.show(truncate=False)

# Stop the SparkSession when done
#spark.stop()


+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|   chegada_prevista|       chegada_real|codigo_autorizacao|codigo_justificativa|codigo_tipo_linha|icao_aerodromo_destino|icao_aerodromo_origem|icao_empresa_aerea|numero_voo|   partida_prevista|       partida_real|situacao_voo|
+-------------------+-------------------+------------------+--------------------+-----------------+----------------------+---------------------+------------------+----------+-------------------+-------------------+------------+
|2021-11-12 08:30:00|2021-11-12 08:30:00|                 0|                 N/A|                X|                  KORD|                 SBGR|               UAL|       844|2021-11-11 22:00:00|2021-11-11 22:14:00|   REALIZADO|
|2021-11-15 08:30:00|2021-11-15 08:30:00|                 0|                 N/A|       

readjust the columns

In [16]:


# Assuming api_dataframe contains the original data

# Define a schema for the JSON data within the "website" column
json_schema = StructType([
    StructField("id", IntegerType()),
    StructField("iata", StringType()),
    StructField("icao", StringType()),
    StructField("name", StringType()),
    StructField("location", StringType()),
    StructField("street_number", StringType()),
    StructField("street", StringType()),
    StructField("city", StringType()),
    StructField("county", StringType()),
    StructField("state", StringType()),
    StructField("country_iso", StringType()),
    StructField("country", StringType()),
    StructField("postal_code", StringType()),
    StructField("phone", StringType()),
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType()),
    StructField("uct", IntegerType()),
    StructField("website", StringType())
])

# Extract JSON data and create a new DataFrame with expanded columns
expanded_data = api_dataframe.withColumn("website_data", from_json(col("website"), json_schema))

# Select the columns you need and filter out rows with "NULL" in the "id" column
expanded_data = expanded_data.filter(col("website_data.id").isNotNull())

# Select the columns you need
expanded_data = expanded_data.select(
    "website_data.id",
    "website_data.iata",
    "website_data.icao",
    "website_data.name",
    "website_data.location",
    "website_data.street_number",
    "website_data.street",
    "website_data.city",
    "website_data.county",
    "website_data.state",
    "website_data.country_iso",
    "website_data.country",
    "website_data.postal_code",
    "website_data.phone",
    "website_data.latitude",
    "website_data.longitude",
    "website_data.uct",
    "website_data.website",
    "icao_code_source"
)

# Show the resulting DataFrame
expanded_data.show(truncate=False)




+----+----+----+----------------------------------------------------------------------+-----------------------------------------------+-------------+--------------------------------------+-----------------+-----------------------+-----------------------+-----------+--------------------+-----------+-----------------+----------+----------+----+-----------------------------------------------------------------------------------------+----------------+
|id  |iata|icao|name                                                                  |location                                       |street_number|street                                |city             |county                 |state                  |country_iso|country             |postal_code|phone            |latitude  |longitude |uct |website                                                                                  |icao_code_source|
+----+----+----+----------------------------------------------------------------------+---------

In [17]:
coalesced_data = expanded_data.coalesce(1)
# Save the expanded_data DataFrame as a Parquet table
coalesced_data.write.mode("overwrite").parquet("/content/test/results.parquet")

# Save the expanded_data DataFrame as a JSON table
coalesced_data.write.mode("overwrite").json("/content/test/results.json")